# **Navigating a World of Change: Insights from Global Socio-Economic and Environmental Data (2001-2020)**

In the first two decades of the 21st century, the world witnessed profound changes across socio-economic and environmental dimensions. An analysis of data from eight diverse countries - Brazil, China, Germany, India, Japan, Kenya, South Africa, and the United States - provides a compelling narrative of these changes and their implications. This article delves into these insights, supported by various data visualizations, to understand better the developmental trajectories, environmental challenges, and technological advancements that have shaped the early part of this century.

In [ ]:
# @title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/dataset.csv')

## Diverse Developmental Stages and Priorities

The journey of nations through developmental stages is uniquely reflected in indicators like access to electricity and immunization rates. Developed countries, such as Germany and the United States, have shown remarkable stability in these areas, a testament to their matured infrastructure and healthcare systems. Conversely, developing nations like India and Kenya exhibit significant improvements, underscoring ongoing efforts in infrastructure development and healthcare. Line graphs and bar charts here reveal the disparities and growth in these fundamental sectors.

In [ ]:
# @title

import plotly.graph_objects as go



# Pivot tables for each indicator
pivot_electricity = data.pivot_table(values='Access to electricity (% of population)',
                                     index='Country Name',
                                     columns='Year')
pivot_immunization = data.pivot_table(values='Immunization, DPT (% of children ages 12-23 months)',
                                      index='Country Name',
                                      columns='Year')

# Create figure with electricity data initially
fig = go.Figure(data=go.Heatmap(
    z=pivot_electricity,
    x=pivot_electricity.columns,
    y=pivot_electricity.index,
    colorscale='Viridis',
    showscale=True
))

# Update layout with dropdown
fig.update_layout(
    title='World Development Indicators',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Country'),
    updatemenus=[dict(
        type='dropdown',
        direction='down',
        x=0.5,
        y=1.15,
        showactive=True,
        active=0,
        buttons=list([
            dict(label='Access to Electricity',
                 method='update',
                 args=[{'z': [pivot_electricity.values]},
                       {'yaxis': {'title': 'Country'},
                        'xaxis': {'title': 'Year'},
                        'title': 'Access to Electricity Over Years by Country'}]),
            dict(label='DPT Immunization Rates',
                 method='update',
                 args=[{'z': [pivot_immunization.values]},
                       {'yaxis': {'title': 'Country'},
                        'xaxis': {'title': 'Year'},
                        'title': 'DPT Immunization Rates Over Years by Country'}])
        ]),
    )]
)

# Show the figure
fig.show()


In [ ]:
# @title
import plotly.graph_objects as go

# Preparing the data
countries = data['Country Name'].unique()
fig = go.Figure()

# Adding traces for each country, but setting them invisible initially
for country in countries:
    country_data = data[data['Country Name'] == country]

    # Access to Electricity trace
    fig.add_trace(go.Scatter(
        x=country_data['Year'],
        y=country_data['Access to electricity (% of population)'],
        name='Access to Electricity (%)',
        visible=False
    ))

    # Immunization trace
    fig.add_trace(go.Scatter(
        x=country_data['Year'],
        y=country_data['Immunization, DPT (% of children ages 12-23 months)'],
        name='DPT Immunization (%)',
        visible=False,
        yaxis='y2'
    ))

# Creating buttons for the dropdown menu
buttons = []
for i, country in enumerate(countries):
    visibility = [False] * len(countries) * 2
    visibility[i*2] = True  # Toggle visibility for Access to Electricity trace
    visibility[i*2 + 1] = True  # Toggle visibility for DPT Immunization trace

    button = dict(label=country,
                  method="update",
                  args=[{"visible": visibility},
                        {"title": f"Access to Electricity and DPT Immunization Over Time in {country}"}])
    buttons.append(button)

# Adding the dropdown menus to the layout
fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons)],
    title="Select a Country to Display Data",
    yaxis=dict(title="Access to Electricity (%)"),
    yaxis2=dict(title="DPT Immunization (%)", overlaying='y', side='right')
)

# Making the first country's data visible initially
fig.data[0].visible = True
fig.data[1].visible = True

fig.show()


In our analysis, we observed significant variations and trends in access to electricity and DPT immunization rates across different countries. Kenya and India, in particular, demonstrated notable improvements in electricity access over the years, reflecting their efforts towards enhancing infrastructure. Developed countries like Germany and the United States maintained a consistent 100% access to electricity throughout the period. In terms of immunization, while Kenya and India showed fluctuations, there was a general trend of improvement. South Africa displayed some variability in its immunization rates, whereas Brazil and China maintained high and stable rates. Developed nations, such as Germany and the United States, consistently exhibited high immunization rates, albeit with slight fluctuations in the case of the United States. These trends highlight the diverse challenges and accomplishments in different regions, underscoring the impact of factors like economic development, government policies, and global health initiatives. The analysis underscores how infrastructure development and healthcare services have evolved differently across countries, influenced by each nation's unique context and the broader global efforts towards improvement in these sectors.

## Environmental Challenges and Responses

The environmental narrative is multifaceted. While China's steep rise in CO2 emissions mirrors its rapid industrialization, the decline in emissions in Germany and the United States suggests effective environmental stewardship. The below visualizations illustrates these emission trends over time, and a scatterplot correlating GDP growth with CO2 emissions offers insights into the complex relationship between economic development and environmental impact.

In [ ]:
# @title
import plotly.graph_objects as go
import numpy as np

# Function to create a trend line
def create_trendline(x, y, name):
    # Fit a linear regression model
    coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)
    trendline_x = np.linspace(min(x), max(x), 100)
    trendline_y = polynomial(trendline_x)
    return trendline_x, trendline_y, name

# Unique list of countries
unique_countries = data['Country Name'].unique()

# Create a figure
fig = go.Figure()

# Add a scatter plot and a trend line for each country
for country in unique_countries:
    country_data = data[data['Country Name'] == country]
    visible = (country == 'India')  # Only the initial country is visible

    # Scatter plot
    fig.add_trace(
        go.Scatter(
            x=country_data['CO2 emissions (metric tons per capita)'],
            y=country_data['GDP growth (annual %)'],
            mode='markers',
            name=f'{country} Data',
            visible=visible
        )
    )

    # Trend line
    trend_x, trend_y, trend_name = create_trendline(
        country_data['CO2 emissions (metric tons per capita)'].dropna(),
        country_data['GDP growth (annual %)'].dropna(),
        f'{country} Trend'
    )
    fig.add_trace(
        go.Scatter(
            x=trend_x,
            y=trend_y,
            mode='lines',
            name=trend_name,
            visible=visible
        )
    )

# Create buttons for the dropdown menu
buttons = [
    dict(
        method="update",
        label=country,
        args=[{"visible": [country == selected_country for selected_country in unique_countries for _ in (0, 1)]},
              {"title": f"CO2 Emissions vs. GDP Growth Over Time for {country}"}]
    ) for country in unique_countries
]

# Add the dropdown menu to the layout
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ],
    title_text="CO2 Emissions vs. GDP Growth Over Time"
)

# Set axis labels
fig.update_xaxes(title_text="CO2 Emissions (Metric Tons per Capita)")
fig.update_yaxes(title_text="GDP Growth (Annual %)")

fig.show()


In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Unique list of countries
unique_countries = data['Country Name'].unique()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for all countries, but set them to be invisible initially
for country in unique_countries:
    country_data = data[data['Country Name'] == country]

    # Add bar chart for CO2 emissions
    fig.add_trace(
        go.Bar(x=country_data['Year'], y=country_data['CO2 emissions (metric tons per capita)'],
               name=f'CO2 Emissions - {country}', visible=(country == 'India')),
        secondary_y=False,
    )

    # Add line chart for GDP growth
    fig.add_trace(
        go.Scatter(x=country_data['Year'], y=country_data['GDP growth (annual %)'],
                   name=f'GDP Growth - {country}', visible=(country == 'India')),
        secondary_y=True,
    )

# Create buttons for the dropdown menu
buttons = []
for country in unique_countries:
    buttons.append(
        dict(
            method="update",
            label=country,
            args=[{"visible": [country == button_country for button_country in unique_countries for _ in (0, 1)]},
                  {"title": f"CO2 Emissions and GDP Growth Over Time for {country}"}]
        )
    )

# Add dropdown menu to the layout
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# Set titles and axis labels
fig.update_layout(
    title_text="CO2 Emissions and GDP Growth Over Time"
)
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="CO2 Emissions (Metric Tons per Capita)", secondary_y=False)
fig.update_yaxes(title_text="GDP Growth (Annual %)", secondary_y=True)

fig.show()


Our analysis revealed a nuanced relationship between CO2 emissions and GDP growth in various countries. Developing nations like Kenya and India exhibited lower levels of CO2 emissions, with India showing a gradual increase alongside its robust economic growth. Brazil's moderate emission levels and fluctuating GDP growth reflect its varying economic activities and policy changes. In stark contrast, China demonstrated a significant rise in CO2 emissions, correlating with its rapid industrialization and consistent high GDP growth. This trend highlights the environmental impact of economic expansion. Developed countries like Germany, Japan, and the United States showed high CO2 emissions, indicative of their industrialized economies. However, their GDP growth rates were more stable and modest, characteristic of mature economies. The data suggests a complex trade-off between economic development and environmental sustainability, especially in rapidly growing economies. This analysis underscores the global challenge of balancing economic growth with the imperative to mitigate environmental impacts, emphasizing the need for sustainable development practices across different stages of national development.

## Economic Growth and Structural Changes

The global economic landscape has seen considerable shifts. All countries experienced growth in GDP per capita, but at varying degrees. This growth, depicted through area charts, highlights the nuances of economic expansion across nations. Furthermore, the changing face of national economies, marked by a shift from industry to services, is effectively portrayed in stacked bar charts, showing the evolving industrial contribution to GDP.

In [ ]:
# @title
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Prepare data for both indicators
indicator1 = 'GDP per capita (current US$)'
indicator2 = 'Industry (including construction), value added (% of GDP)'

# Create a figure with secondary y-axis
fig_area = make_subplots(specs=[[{"secondary_y": True}]])

# Determine unique countries and years
countries = data['Country Name'].unique()
years = data['Year'].unique()

# Add traces for the first indicator
for country in countries:
    country_data = data[data['Country Name'] == country]
    fig_area.add_trace(go.Scatter(x=country_data['Year'],
                                  y=country_data[indicator1],
                                  name=country,
                                  stackgroup='one', # stack traces
                                  visible=True), secondary_y=False)

# Add traces for the second indicator
for country in countries:
    country_data = data[data['Country Name'] == country]
    fig_area.add_trace(go.Scatter(x=country_data['Year'],
                                  y=country_data[indicator2],
                                  name=country,
                                  stackgroup='two', # stack traces
                                  visible=False), secondary_y=True)

# Add dropdown
fig_area.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label=indicator1,
                     method="update",
                     args=[{"visible": [True]*len(countries) + [False]*len(countries)}]), # Show first indicator
                dict(label=indicator2,
                     method="update",
                     args=[{"visible": [False]*len(countries) + [True]*len(countries)}]), # Show second indicator
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

fig_area.show()


In [ ]:
# @title
import plotly.graph_objs as go
import plotly.express as px

filtered_data = data[data["Country Name"].isin(countries)]

# Define a list of distinct colors for the countries
colors = px.colors.qualitative.Plotly

# Mapping each country to a color
color_map = {country: colors[i % len(colors)] for i, country in enumerate(sorted(filtered_data['Country Name'].unique()))}

# Creating a figure with a subplot for each year
fig = go.Figure()

# Adding one trace for each year
for year in sorted(filtered_data['Year'].unique()):
    # Filter data for each year
    year_data = filtered_data[filtered_data['Year'] == year]
    fig.add_trace(
        go.Bar(
            x=year_data['Country Name'],
            y=year_data['Industry (including construction), value added (% of GDP)'],
            name=str(year),
            visible=(year == sorted(filtered_data['Year'].unique())[0]),  # Only the first year is visible initially
            marker=dict(color=[color_map[country] for country in year_data['Country Name']])  # Assigning colors
        )
    )

# Update layout to add dropdown
fig.update_layout(
    title="Industry Contribution to GDP by Country and Year",
    updatemenus=[
        {
            "buttons": [
                {
                    "label": str(year),
                    "method": "update",
                    "args": [{"visible": [year == y for y in sorted(filtered_data['Year'].unique())]},
                             {"title": f"Industry Contribution to GDP in {year}"}]
                } for year in sorted(filtered_data['Year'].unique())
            ],
            "direction": "down",
            "showactive": True,
        }
    ]
)

fig.show()



In the analysis of the GDP per capita and the industry (including construction) value added as a percentage of GDP over a 20-year period, several key insights emerged:

GDP per Capita Trends: The GDP per capita, which reflects economic output per person, generally exhibited an upward trajectory across most countries. This indicates broad economic growth over the two decades. However, there was notable variation both in the growth rates and the absolute GDP per capita figures, highlighting the diverse economic landscapes and development stages of different countries.

Industry Contribution to GDP: The contribution of the industrial sector, including construction, to the overall GDP displayed more variability. Some countries maintained a relatively steady percentage of GDP from industry, whereas others experienced more pronounced fluctuations. These trends could be attributed to a range of factors, including shifts in economic policy, changes in industrial capacity, and broader global economic influences.

Comparative Analysis: The comparative analysis revealed significant differences in economic indicators across countries. These disparities underscore the unique economic conditions, resource endowments, and policy environments that shape each country's economic trajectory.

## Technological Adoption and Connectivity

The technology revolution is perhaps most striking in the meteoric rise of mobile cellular subscriptions, particularly in developing countries. Choropleth map here do not just represent numbers; they signify a leap towards greater connectivity and the opening of new avenues in communication and commerce.

In [ ]:
# @title
mobile_subscriptions = data[['Country Name', 'Country Code', 'Year', 'Mobile cellular subscriptions (per 100 people)']]

# Creating the animated choropleth map
fig = px.choropleth(mobile_subscriptions,
                    locations="Country Code",
                    color="Mobile cellular subscriptions (per 100 people)",
                    hover_name="Country Name",
                    animation_frame="Year",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    projection="natural earth",
                    title="Mobile Cellular Subscriptions (per 100 people) Over 20 Years")

# Show the figure
fig.show()

In the analysis of mobile cellular subscriptions (per 100 people) over the past 20 years, we observed significant differences in growth rates across various countries. Developing nations like India and China experienced tremendous growth, with India's subscriptions skyrocketing by approximately 13,530.84% and China's by around 959.85%. This surge can be attributed to the rapid expansion of mobile technology and its increasing affordability in these regions. On the other hand, developed countries such as Germany and Japan exhibited more moderate growth rates, at 87.19% and 164.49% respectively, likely indicating a saturation point or already high subscription levels at the beginning of the period. Emerging markets like Brazil also showed substantial growth, with an increase of about 498.55%, reflecting the strong adoption of mobile technology. Overall, these trends highlight the varying stages of mobile technology adoption across different economies, from rapid expansion in developing countries to more stabilized growth in developed nations.

## **Health and Demographic Shifts**

Health and demographic trends offer hopeful signs. The decline in adolescent fertility rates across most countries, as shown in line charts, speaks to changing social norms and improved access to education and healthcare. Meanwhile, bubble charts illustrating immunization rates juxtaposed with population sizes underscore the global emphasis on child health.

In [ ]:
# @title
fig_line_fertility = px.line(filtered_data, x="Year", y="Adolescent fertility rate (births per 1,000 women ages 15-19)",
                             color="Country Name", title="Adolescent Fertility Rate Over Time by Country")
fig_line_fertility.show()


In [ ]:
# @title
fig_bubble_immunization = px.scatter(filtered_data, x="Year", y="Immunization, DPT (% of children ages 12-23 months)",
                                     size="Adolescent fertility rate (births per 1,000 women ages 15-19)", color="Country Name",
                                     title="Immunization Rates and Fertility Rate Over Time by Country")
fig_bubble_immunization.show()


The analysis of the trends in "Immunization, DPT (% of children ages 12-23 months)" and "Adolescent fertility rate (births per 1,000 women ages 15-19)" revealed the following insights. The immunization rates across these countries generally remained high and stable or exhibited a slight increase, suggesting effective and sustained child healthcare and vaccination programs. On the other hand, the adolescent fertility rates showed a consistent decline, indicative of advancements in education, better access to healthcare, and improved reproductive rights for young women. The rate and extent of this decline varied among countries, reflecting the diverse socio-economic and cultural environments. These trends collectively highlight positive developments in global health, particularly in child healthcare and women's reproductive health. However, the variation in rates also underscores the ongoing challenges in different regions and the need for tailored approaches to address specific health and social issues.

## Agricultural Trends

Agricultural land use, pivotal for both economic and environmental reasons, has seen varying trends. Scatter plot could provide a geographical perspective on these changes, over time reflect shifts in agricultural practices and land use policies.

In [ ]:
# @title
import plotly.express as px
import pandas as pd

# Assuming 'data' is your DataFrame
# Example: Plotting 'Agricultural land (% of land area)' against 'Year' for different countries

# You might need to filter or process your data depending on what you want to visualize
scatter_data = data[['Country Name', 'Year', 'Agricultural land (% of land area)']]

# Creating the scatter plot
fig = px.scatter(scatter_data,
                 x='Year',
                 y='Agricultural land (% of land area)',
                 color='Country Name',  # Different colors for different countries
                 title='Agricultural Land (% of Land Area) Over Years by Country')

# Show the plot
fig.show()


The analysis of the "Agricultural land (% of land area)" indicator over a 20-year period in a sample of countries revealed diverse trends in the use of land for agricultural purposes. Some countries showed an increasing trend in agricultural land use, potentially driven by the expansion of farming activities to meet the demands of a growing population or to capitalize on export opportunities. In contrast, other countries exhibited a decrease in agricultural land, which could be attributed to factors such as urbanization, industrial development, or a shift towards more efficient farming practices. Additionally, some countries demonstrated stable trends, indicating a possible equilibrium between agricultural expansion and land conservation or efficient utilization of existing agricultural resources. These trends reflect the complex interplay of demographic changes, economic development, environmental policies, and land management practices. They highlight the importance of understanding country-specific contexts and have significant implications for food security, environmental sustainability, and economic development in these regions.

## Conclusion: Towards a Holistic Understanding

The data from 2001 to 2020 paints a picture of a world in flux, grappling with the challenges of development, environmental sustainability, and rapid technological change. While developed countries display stability and maturity in basic services, developing nations show dynamic growth and improvement in these areas. The environmental narrative remains complex, with urgent calls for sustainable practices. The economic landscape reveals steady growth, albeit with structural shifts, and the technological revolution continues unabated, reshaping societies.

 The early 21st century has been a period of significant transformation. Understanding these trends is crucial for policymakers, businesses, and individuals as they navigate a rapidly changing world. This understanding, rooted in data and enhanced by visual storytelling, is not just about comprehending the past but about preparing for the future.

From the overall analysis of the dataset covering various indicators across multiple countries over a 20-year period, we can draw several key conclusions:



*   Progress in Access to Basic Services: Many countries have shown significant progress in improving access to essential services like electricity. This improvement is indicative of developmental strides and enhanced living standards.

*   Economic Growth: There's a general upward trend in economic indicators such as GDP per capita. This growth signifies increased economic activity and potentially improved prosperity in the countries analyzed.


*   Health and Social Development: High and improving rates of immunization and declining adolescent fertility rates reflect progress in healthcare and social development. These trends suggest better health outcomes, increased awareness, and access to healthcare services.

*   Environmental Challenges: The varying trends in CO2 emissions and the correlation between economic growth and environmental impact highlight a crucial challenge. While economic development is crucial, it often comes with increased environmental pressures, particularly in terms of carbon emissions. This presents a need for sustainable development strategies that balance economic growth with environmental conservation.

*   Diverse Development Stages: The countries in the dataset are at various stages of development, with diverse trajectories in economic, social, and environmental indicators. This diversity underscores the importance of context-specific policies and interventions.


*   Interplay Between Different Indicators: The correlations observed between different indicators, such as between economic growth and CO2 emissions or between GDP per capita and health indicators, suggest a complex interplay between various aspects of development. These relationships underscore the multifaceted nature of development, where progress in one area can impact others.

*   Need for Sustainable and Inclusive Growth: The analysis indicates a need for policies and practices that promote sustainable and inclusive growth. This includes addressing environmental concerns while pursuing economic development and ensuring that the benefits of growth are broadly shared across society.


In summary, the analysis paints a picture of progressing development across different nations, marked by economic growth, improved access to essential services, and better health outcomes. However, it also highlights the ongoing challenges of sustainable development, particularly in balancing economic growth with environmental sustainability and ensuring equitable social progress.






## Take away



*   Development Trends: There is evidence of development progress over the 20-year period, as seen in increased access to electricity, higher GDP per capita, and improved health indicators like immunization rates.

*   Economic Growth vs Environmental Impact: The positive correlation between economic growth and CO2 emissions highlights the ongoing challenge of achieving economic development while managing environmental impacts.



*  Social Indicators and Development: The decline in adolescent fertility rates and the positive trends in health and economic indicators suggest overall social progress in the countries studied.





